<a href="https://colab.research.google.com/github/behrangEhi/ML-DL-Projects/blob/main/svm_dual_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Implementing Dual Support Vector Machine (SVM) Optimization in Google Colab

Introduction:
This Google Colab notebook demonstrates the implementation of a dual optimization algorithm for training a Support Vector Machine (SVM) classifier. The code provides functions for creating a normalized and centered kernel, as well as the core SVM dual optimization routine.

The main objectives of this Colab are:

1. To walk through the process of implementing a dual SVM optimization algorithm in Python.
2. To show how to leverage the powerful computational resources and libraries available in Google Colab to train and evaluate SVM models.
3. To provide a reusable code template that can be adapted and extended for various SVM-based machine learning tasks.

By working through this Colab, you will gain a deeper understanding of the dual SVM optimization approach and how to apply it using Python and the libraries available in the Colab environment.

Let's get started!

In [ ]:
import numpy as np
import pandas as pd

def create_normalized_centered_kernel(K):
    n = K.shape[0]
    centered_K = (np.identity(n) - (1/n)*np.ones((n,n))).dot(K).dot(np.identity(n) - (1/n)*np.ones((n,n)))
    W = np.diag(np.diag(centered_K))
    normalized_centered_K = np.power(W,-1/2, where=(W!=0)).dot(centered_K).dot(np.power(W,-1/2, where=(W!=0)))
    return normalized_centered_K

def SVM_DUAL(D,y,Kernel,C,epsilon,loss):
    n = D.shape[0]

    if loss=="hinge":
        K = Kernel
    elif loss=="quadratic":
        K = Kernel + (1/(2*C))*np.eye(n)

    K_q = K + np.ones(K.shape)
    eta = 1/np.diagonal(K_q)
    t = 0
    alphas = [np.zeros((n,1))]
    while True:
        alpha = np.copy(alphas[t])
        for k in range(n):
            alpha[k] = alpha[k] + eta[k]*(1-y[k]*np.sum(np.multiply(alpha,np.multiply(y,K_q[:,k]))))
            if alpha[k] < 0:
                alpha[k] = 0
            if loss=="hinge" and alpha[k] > C:
                alpha[k] = C
        alphas.append(alpha)
        t+=1
        if np.linalg.norm(alphas[-1]-alphas[-2]) <= epsilon:
            break
    yhat = np.zeros((n,1))
    for k in range(n):
        yhat[k] = np.sum(np.multiply(alphas[-1],np.multiply(y,K_q[:,k])))
    yhat = np.sign(yhat)

    return alphas[-1],yhat

def print_results(alpha,yhat,kernel_type):
    print(kernel_type,":")
    num = 0
    for i in range(len(alpha)):
        if alpha[i][0]>0:
            num+=1
            print("i = ",i+1,", alpha_i = ",alpha[i][0])
    print("number of support vectors = ",num)
    print("accuracy = ",np.sum([yhat[i][0]==y[i] for i in range(len(y))])/len(y))

C=10
epsilon = 0.0001

In [ ]:
data = pd.read_csv("iris-slwc.txt",names=["length","width","class"])
D = data[["length","width"]]
y = data["class"].to_numpy()

In [ ]:
linear_Kernel = np.array(D.dot(D.T).pow(1))
alpha,yhat = SVM_DUAL(D,y,create_normalized_centered_kernel(linear_Kernel),C,epsilon,"hinge")
print_results(alpha,yhat,"linear kernel")

In [ ]:
homogeneous_quadrat_Kernel = np.array(D.dot(D.T).pow(2))
alpha,yhat = SVM_DUAL(D,y,create_normalized_centered_kernel(homogeneous_quadrat_Kernel),C,epsilon,"hinge")
print_results(alpha,yhat,"homogeneous quadrat")

In [ ]:
n = D.shape[0]
inhomogeneous_quadrat_Kernel = np.array((D.dot(D.T)+1*np.ones((n,n))).pow(2))
alpha,yhat = SVM_DUAL(D,y,create_normalized_centered_kernel(inhomogeneous_quadrat_Kernel),C,epsilon,"hinge")
print_results(alpha,yhat,"inhomogeneous quadrat")

In [ ]:
data = pd.read_csv("iris-PC.txt",delim_whitespace=True,names=["length","width","class"])
D = data[["length","width"]]
y = data["class"].to_numpy()

In [ ]:
linear_Kernel = np.array(D.dot(D.T).pow(1))
alpha,yhat = SVM_DUAL(D,y,create_normalized_centered_kernel(linear_Kernel),C,epsilon,"hinge")
print_results(alpha,yhat,"linear kernel")

In [ ]:
homogeneous_quadrat_Kernel = np.array(D.dot(D.T).pow(2))
alpha,yhat = SVM_DUAL(D,y,create_normalized_centered_kernel(homogeneous_quadrat_Kernel),C,epsilon,"hinge")
print_results(alpha,yhat,"homogeneous quadrat")

In [ ]:
n = D.shape[0]
inhomogeneous_quadrat_Kernel = np.array((D.dot(D.T)+1*np.ones((n,n))).pow(2))
alpha,yhat = SVM_DUAL(D,y,create_normalized_centered_kernel(inhomogeneous_quadrat_Kernel),C,epsilon,"hinge")
print_results(alpha,yhat,"inhomogeneous quadrat")